In [9]:
import pandas as pd
import numpy as np
df = pd.read_csv('./dataset/reorder_counts.csv')
df

,user_id,product_id,count
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3
...,...,...,...
13307948,206209,43961,3
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [2]:
filteronce = df[df['count'] > 2]
filteronce 

,user_id,product_id,count
0,1,196,10
1,1,10258,9
3,1,12427,10
4,1,13032,3
8,1,25133,8
...,...,...,...
13307927,206209,25837,3
13307939,206209,38167,4
13307945,206209,40992,3
13307946,206209,41213,7


In [3]:
#Find every item pair order by same user
joined = filteronce.merge(filteronce, on= 'user_id', how='outer')
joined[joined['user_id']==1]

,user_id,product_id_x,count_x,product_id_y,count_y
0,1,196,10,196,10
1,1,196,10,10258,9
2,1,196,10,12427,10
3,1,196,10,13032,3
4,1,196,10,25133,8
5,1,196,10,46149,3
6,1,10258,9,196,10
7,1,10258,9,10258,9
8,1,10258,9,12427,10
9,1,10258,9,13032,3


In [4]:
filterDuplicates = joined[joined['product_id_x'] > joined['product_id_y']]
filterDuplicates[filterDuplicates['user_id']==1]

,user_id,product_id_x,count_x,product_id_y,count_y
6,1,10258,9,196,10
12,1,12427,10,196,10
13,1,12427,10,10258,9
18,1,13032,3,196,10
19,1,13032,3,10258,9
20,1,13032,3,12427,10
24,1,25133,8,196,10
25,1,25133,8,10258,9
26,1,25133,8,12427,10
27,1,25133,8,13032,3


In [14]:
#Make the product_id pairs as key
grouped = filterDuplicates.groupby(['product_id_x','product_id_y'])
size = grouped.size()
size

product_id_x  product_id_y
10            2               1
25            10              5
              23              1
26            12              1
28            25              1
                             ..
49688         47122           1
              47479           1
              47629           1
              48253           1
              48527           1
Length: 17955567, dtype: int64

In [40]:
filtersize = size[size > 2]
allpair = pd.DataFrame({'count' : filtersize}).reset_index()
allpair.index = np.arange(1, len(allpair)+1)
allpair.reset_index(level=0, inplace=True)
allpair = allpair.rename(columns={'index':  'itemPair_id'})
allpair

,itemPair_id,product_id_x,product_id_y,count
0,1,25,10,5
1,2,34,25,3
2,3,45,10,6
3,4,45,23,3
4,5,45,25,7
...,...,...,...,...
3704058,3704059,49686,46979,5
3704059,3704060,49686,47626,7
3704060,3704061,49686,47766,5
3704061,3704062,49686,49520,3


In [31]:
allpair.to_csv('./dataset/itemPair_to_id.csv', index=False)

In [41]:
merged = allpair.merge(filterDuplicates, on= ['product_id_x', 'product_id_y'], how='left')
merged

,itemPair_id,product_id_x,product_id_y,count,user_id,count_x,count_y
0,1,25,10,5,10909,9,3
1,1,25,10,5,50598,11,13
2,1,25,10,5,101124,5,3
3,1,25,10,5,153284,4,5
4,1,25,10,5,190521,3,3
...,...,...,...,...,...,...,...
46391086,3704062,49686,49520,3,102268,3,29
46391087,3704063,49686,49683,4,10660,4,4
46391088,3704063,49686,49683,4,16163,4,24
46391089,3704063,49686,49683,4,174261,16,13


In [43]:
#new_df = merged.drop(['count', 'user_id', 'product_id_x', 'product_id_y'], axis=1)
new_df = merged.drop(['count', 'user_id', 'itemPair_id'], axis=1)
new_df

,product_id_x,product_id_y,count_x,count_y
0,25,10,9,3
1,25,10,11,13
2,25,10,5,3
3,25,10,4,5
4,25,10,3,3
...,...,...,...,...
46391086,49686,49520,3,29
46391087,49686,49683,4,4
46391088,49686,49683,4,24
46391089,49686,49683,16,13


In [44]:
new_df.to_csv('./dataset/reorder_itemPair_v3.csv', index=False)